Импорт необходимых библиотек

In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from category_encoders.target_encoder import TargetEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

pd.options.display.max_columns = 100

Загрузка обучающего и тестового датасетов

Разметка полей. def - таргет для модели. id - идентификатор. Все остальные поля - факторы для обучения модели

In [2]:
train = pd.read_csv('data/logreg_train.csv')
test = pd.read_csv('data/logreg_test.csv')

In [21]:
purpose = [x for x in train.columns if x.startswith('purpose_')]
subgrade = [x for x in train.columns if x.startswith('sub_grade_')]
home_ownership = [x for x in train.columns if x.startswith('home_ownership_')]

train['purpose'] = train[purpose].idxmax(axis=1)
# train.drop(columns=purpose, axis=1, inplace=True)
train['subgrade'] = train[subgrade].idxmax(axis=1)
# train.drop(columns=subgrade, axis=1, inplace=True)
train['home_ownership'] = train[home_ownership].idxmax(axis=1)
# train.drop(columns=home_ownership, axis=1, inplace=True)

test['purpose'] = test[purpose].idxmax(axis=1)
# test.drop(columns=purpose, axis=1, inplace=True)
test['subgrade'] = test[subgrade].idxmax(axis=1)
# test.drop(columns=subgrade, axis=1, inplace=True)
test['home_ownership'] = test[home_ownership].idxmax(axis=1)
# test.drop(columns=home_ownership, axis=1, inplace=True)

In [24]:
cols = [
    'purpose', 'home_ownership', 'subgrade', 'emp_length', 'term', 'delinq_2yrs', 'num_accts_ever_120_pd', 'acc_open_past_24mths',
]
te = TargetEncoder(cols=cols)
sample_df = te.fit_transform(train[cols], train['def'])
test_sample_df = te.transform(test[cols])

sample_df.columns = [f"{x}_encoded" for x in sample_df.columns]
test_sample_df.columns = [f"{x}_encoded" for x in test_sample_df.columns]

d:\kaggle\sber_plus_nsu\.venv\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [29]:
train = pd.concat([train, sample_df], axis=1)
test = pd.concat([test, test_sample_df], axis=1)

In [30]:
targ_cols = [i for i in train.columns if i not in ['def', 'id']  and 'sub_grade' not in i]

X_train = train[targ_cols]
X_test = test[targ_cols]

y_train = train["def"]

Обучение модели

In [6]:
knn_params = {
    "fit_intercept": True,
    "max_iter": 300,
    "penalty": "l1",
    "random_state": 123,
    "solver": "liblinear"
}

model = LogisticRegression(**knn_params)

cols = [
    'percent_per_year',
    'percent',
    'term',
    'loan_income_ratio',
    'psk',
    'additional_dti',
    
    'purpose_small_business',
    'purpose_home_improvement',
    'purpose_major_purchase',
    'purpose_medical',
    'purpose_vacation',
    'purpose_wedding',
    'purpose_car',
    
    'acc_open_past_24mths',
    'num_accts_ever_120_pd',
    
    'home_ownership_MORTGAGE',
    'home_ownership_RENT',
    
    'installment',
    'is_first_loan',
    'avg_cur_bal',
    'is_emp_length_unknown',
    'even_salary',
]

skf = StratifiedKFold(n_splits=5, random_state=16, shuffle=True)
scores = cross_val_score(model, X_train[cols], y_train, cv=skf, scoring='roc_auc')
print(np.mean(scores), np.std(scores))

0.6859147742494416 0.007224020156917809


In [39]:
def do_lr_cv(x_train, x_test, y_train, features, params):
    oof_result = pd.Series([0] * x_train.shape[0], index=x_train.index)
    result = [0] * x_test.shape[0]
    scores = []
    for ind_trn, ind_test in skf.split(x_train, y_train):
        X_train_df = x_train[features].iloc[ind_trn]
        X_valid_df = x_train[features].iloc[ind_test]
        y_train_df = y_train.iloc[ind_trn]
        y_valid_df = y_train.iloc[ind_test]

        model = LogisticRegression(**params)
        model.fit(X_train_df, y_train_df)

        scores.append(roc_auc_score(y_valid_df, model.predict_proba(X_valid_df)[:, 1]))
        result += model.predict_proba(x_test[features])[:, 1:].flatten()/5
        oof_result.loc[X_valid_df.index] = model.predict_proba(X_valid_df)[:, 1]
    return scores, result, oof_result, model

In [37]:
params = {
    "fit_intercept": True,
    "max_iter": 300,
    "penalty": "l1",
    "random_state": 123,
    "solver": "liblinear"
}

init_cols = [
    'dti',
    'inq_last_6mths',
#     'mths_since_recent_inq',
#     'num_accts_ever_120_pd',
    'acc_open_past_24mths',
    'tot_hi_cred_lim',
#     'purpose',
#     'home_ownership',
#     'psk',
    'percent',
#     'loan_income_ratio',
#     'is_emp_length_unknown',
#     'funded_amnt_mod_100', 
    'annual_inc',
    'purpose_encoded',
    'additional_dti',
    
]

rs = {}

skf = StratifiedKFold(n_splits=5, random_state=18, shuffle=True)
for feat in targ_cols:
    if feat in init_cols or feat in ['purpose', 'home_ownership', 'subgrade',]:
        continue

    scores, result, oof_result, _ = do_lr_cv(X_train, X_test, y_train, init_cols + [feat], params)
    print(feat, np.mean(scores), np.std(scores))
    rs[feat] = [np.mean(scores), np.std(scores)]

installment 0.6820934343945908 0.004996731905542112
funded_amnt 0.6823073493775065 0.005321250650089259
emp_length 0.6817605715341075 0.005418849037518057
term 0.681951903622625 0.006148596187784299
mths_since_recent_inq 0.6817169449015049 0.005494543839893528
delinq_2yrs 0.6819668364969697 0.005457515146146881
chargeoff_within_12_mths 0.6817190146828477 0.00568764570023921
num_accts_ever_120_pd 0.6821317812133897 0.0053336520604742605
num_tl_90g_dpd_24m 0.6819662912175197 0.005567713651968123
avg_cur_bal 0.6817300111140836 0.005447236262194565
delinq_amnt 0.6816908628558246 0.005576146360252083
purpose_car 0.6817107273065135 0.005502543253482141
purpose_credit_card 0.6819521260576272 0.005557677185498784
purpose_debt_consolidation 0.6820672653449406 0.00589130105835501
purpose_home_improvement 0.6817579698316558 0.005795789437916118
purpose_house 0.6816104629032294 0.005559480228342022
purpose_major_purchase 0.6816789555659512 0.005653711166489751
purpose_medical 0.6816191571945975 0.

In [38]:
pd.DataFrame(rs).T.sort_values(0).tail(10)

,0,1
home_ownership_encoded,0.682434,0.005573
emp_length_encoded,0.682456,0.005542
home_ownership_RENT,0.682484,0.005696
funded_amnt_sqrt,0.682514,0.005647
is_not_sber_client,0.682643,0.006396
funded_amnt_mod_100,0.682712,0.005500
subgrade_encoded,0.683100,0.006689
num_accts_ever_120_pd_encoded,0.683225,0.005632
annual_inc_log,0.683275,0.005685
annual_inc_sqrt,0.683290,0.005704


In [36]:
pd.DataFrame(rs).T.sort_values(0).tail(10)

,0,1
funded_amnt,0.681615,0.005628
is_not_sber_client,0.681676,0.006520
percent_per_year,0.681747,0.006793
funded_amnt_mod_100,0.681749,0.005524
inq_last_6mths,0.681826,0.005551
funded_amnt_sqrt,0.681877,0.005981
annual_inc_log,0.682287,0.005937
annual_inc_sqrt,0.682298,0.005939
num_accts_ever_120_pd_encoded,0.682354,0.005860
subgrade_encoded,0.682734,0.006911


In [111]:
remove_list = [
    'purpose', 'home_ownership', 'subgrade',
    'additional_dti', 'delinq_2yrs_encoded',
    'annual_inc_sqrt', 'dti', 'dti_sqrt',
    'emp_length_encoded', 'even_installment',
    'funded_amnt', 'home_ownership_NONE',
    'home_ownership_OTHER',
    'home_ownership_encoded', 'installment',
    'purpose_encoded', 'purpose_home_improvement',
    'purpose_renewable_energy', 'term_encoded',
    'emp_length', 'funded_amnt_sqrt', 'home_ownership_RENT',
    'annual_inc_log', 'waste_to_installment_ratio',
    'delinq_amnt', 'funded_amnt_mod_1000',
    'term', 'funded_amnt_mod_500', 'tot_hi_cred_lim',
    'num_tl_90g_dpd_24m', 'funded_amnt_mod_100',
    'purpose_other', 'acc_open_past_24mths', 'mths_since_recent_inq',
    'tot_hi_cred_lim_funded_amnt_ratio', 'acc_open_past_24mths_ratio', 'funded_amnt_mod_10',
    'num_accts_ever_120_pd',
    
#     'dti_ratio_min', 'purpose_medical', 'delinq_2yrs',
#     'is_common_amount', 'inq_last_6mths', 'purpose_moving',
    
]

params = {
    "fit_intercept": True,
    "max_iter": 300,
    "penalty": "l1",
    "random_state": 123,
    "solver": "liblinear",
    "C": 1.6
}

cols = targ_cols.copy()

for r_c in remove_list:
    cols.remove(r_c)

scores, result, oof_result, model = do_lr_cv(
    X_train,
    X_test,
    y_train,
    cols,
    params,
)

In [112]:
# C 1
np.mean(scores), np.std(scores), roc_auc_score(y_train, oof_result)

(0.6886618945327244, 0.00766254320548039, 0.6885790591148633)

Рассчет метрики на обучающей модели

In [114]:
roc_auc_score(y_train, model.predict_proba(X_train[cols])[:, 1])

0.6921836062696067

Применение модели к тестовой выборке

In [115]:
test['prob'] = result

Выгрузка решения для коммита в соревновании

In [116]:
test[['id', 'prob']].to_csv('lr_submission.csv', index=False)

In [30]:
test[['id', 'prob']]

,id,prob
0,0,0.129688
1,1,0.141625
2,2,0.235351
3,3,0.161320
4,4,0.159157
...,...,...
29995,29995,0.119276
29996,29996,0.044804
29997,29997,0.278472
29998,29998,0.033987


In [117]:
test[['id', 'prob']]

,id,prob
0,0,0.113271
1,1,0.120397
2,2,0.214433
3,3,0.150375
4,4,0.129360
...,...,...
29995,29995,0.119279
29996,29996,0.051602
29997,29997,0.270260
29998,29998,0.029760
